# **Sentiment Index Generation: Llama**

# **All Installations**

In [1]:
%pip install transformers datasets peft accelerate evaluate torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00


In [2]:
%pip install python-dotenv

In [3]:
import os
import platform

def data_download(file_to_download, gdrive_code, OS, uncompress = True):
  if not os.path.exists(file_to_download):
    os.system('gdown --id "'+gdrive_code+'" --output '+file_to_download)
    if OS == "Linux" and uncompress:
        os.system('unzip -o -n "./'+file_to_download+'" -d "./"')
    return True
  else:
    return None

OS = platform.system()

out = data_download("./Sentiment_Dataset.zip", "1OEQpkzN6HMcZc3yPg8G0ahvh6lOCMlIR", OS)

# **All Imports**

In [4]:
#General
import torch
import random
import numpy as np
import pandas as pd
from dotenv import load_dotenv

# LLMs
import evaluate
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model

# Machine Learning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# **All Globals**

In [5]:
seed = 42
dataset_limit = 100
llm_model_parameters_number = '1B'                # number of billion parameters of the LLM mode, 1B is the smaller llama3.2 1 billion of parameters
model_name = f"meta-llama/Llama-3.2-{llm_model_parameters_number}"
dataset_path = "./Sentiment_Dataset/"

# **Initializations**

In [6]:
os.environ["WANDB_MODE"] = "disabled"

!mv "{dataset_path}.env" ./
random.seed(seed)
np.random.seed(seed)

load_dotenv()

True

# **Functions Definition**

In [7]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

# **Dataset Reading**

In [47]:
irony_df = pd.read_csv("/content/Sentiment_Dataset/irony_dataset.txt", header=None).rename(columns={0: 'text'})["text"].to_frame()

In [8]:
df = pd.read_csv("/content/Sentiment_Dataset/training_set.csv", sep = ";")
df = df[["text", "pol"]].dropna()         # select the columns interesting for us and drop rows with NaN

In [9]:
if dataset_limit is not None:
    df = df.sample(dataset_limit).reset_index(drop=True)

In [10]:
print("Numnber of data-points:", len(df))

Numnber of data-points: 100


In [11]:
df.head()

,text,pol
0,Arbasina/Giavazzi sul Corriere a proposito del...,0.0
1,il trading è un amp azienda ecco come risparmi...,1.0
2,"RT TAG mi hanno unfollowato in massa, mortacci...",2.0
3,modalità prezzi e biglietti per la gara al man...,0.0
4,donnastef shop t shirt gonna borsa spedizione ...,0.0


1: 'Positive', 2: 'Negative', 0: 'Neutral'

In [12]:
print("Number of classes", df["pol"].nunique())
print("Types of classes", df["pol"].unique())

Number of classes 3
Types of classes [0. 1. 2.]


# **Dataset Recoding**

In [13]:
df['pol'] = df['pol'].map({1: 'Positive', 2: 'Negative', 0: 'Neutral'})
df = df.rename(columns={'pol': 'label'})

In [14]:
df.head()

,text,label
0,Arbasina/Giavazzi sul Corriere a proposito del...,Neutral
1,il trading è un amp azienda ecco come risparmi...,Positive
2,"RT TAG mi hanno unfollowato in massa, mortacci...",Negative
3,modalità prezzi e biglietti per la gara al man...,Neutral
4,donnastef shop t shirt gonna borsa spedizione ...,Neutral


# **Label Conversion**

In [15]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
np.save("labels_mapping.npy", label_mapping)
print("Label mapping:", label_mapping)

Label mapping: {'Negative': np.int64(0), 'Neutral': np.int64(1), 'Positive': np.int64(2)}


# **Train-Test Split**

In [16]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# **Dataset Conversion for Transformers**

In [17]:
irony_dataset = Dataset.from_pandas(irony_df)
tokenized_irony = irony_dataset.map(tokenize_function, batched=True)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
unique_labels = train_dataset.unique('label')
num_labels = len(unique_labels)
print("Number of Labels withing the Training Set:", len(unique_labels))

Number of Labels withing the Training Set: 3


In [18]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 80
})

In [19]:
test_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 20
})

# **Tokenization and Padding**

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [21]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

## **Load the LLM Model**

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **Definition of QLora**

In [23]:
# === 7. Applica LoRA (fine-tuning efficiente) ===
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, peft_config)

 # **Metrics Definition**

In [24]:
accuracy = evaluate.load("accuracy")

In [25]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    return accuracy.compute(predictions=preds, references=labels)

In [26]:
training_args = TrainingArguments(
    output_dir="./llama3_finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    report_to=[]  # Disactivate the W&B
)

In [27]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3389762461.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [28]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.105469,0.200000
2,1.916400,1.954297,0.200000
3,1.651500,1.901172,0.150000


TrainOutput(global_step=120, training_loss=1.7815104166666667, metrics={'train_runtime': 123.2711, 'train_samples_per_second': 1.947, 'train_steps_per_second': 0.973, 'total_flos': 359059182059520.0, 'train_loss': 1.7815104166666667, 'epoch': 3.0})

In [29]:
results = trainer.evaluate()
print(f"\n✅ Accuracy sul test set: {results['eval_accuracy']:.4f}")


✅ Accuracy sul test set: 0.1500


In [52]:
predictions = trainer.predict(tokenized_irony)
predicted_labels = predictions.predictions.argmax(-1)
label_mapping = np.load("labels_mapping.npy", allow_pickle=True).item()
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
predicted_sentiment = [reverse_label_mapping[label] for label in predicted_labels]
irony_df['predicted_sentiment'] = predicted_sentiment
display(irony_df.head())

Map:   0%|          | 0/71 [00:00<?, ? examples/s]